<a href="https://colab.research.google.com/github/TheHunter281956/Di_no/blob/Colab/di_no.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
import requests

auth.authenticate_user()

def run_notebook():
    requests.post("https://www.googleapis.com/colab/v1/sessions")

run_notebook()

In [ ]:
!apt update && apt upgrade -y
!apt install -y libgtk-4-dev libgraphene-1.0-0 libgstreamer-gl1.0-0 \
  libgstreamer-plugins-bad1.0-0 libavif13 libharfbuzz-icu0 libenchant-2-2 \
  libsecret-1-0 libhyphen0 libmanette-0.2-0 woff2

!pip install playwright
!playwright install --with-deps

In [ ]:
import asyncio
import nest_asyncio
from datetime import datetime, timedelta
from playwright.async_api import async_playwright
from playwright._impl._errors import TargetClosedError

nest_asyncio.apply()

async def scrape_counters_for_date(page, date_str):
    target_url = (
        "https://app.di.no/app/play/edg/edg/info?"
        f"_edgForm.selectedRouteId=174809&_edgForm.selectedDateString={date_str}"
        "&_edgForm.mode=Normal&_edgForm.selectedLanguageCode=no&_edgForm.showDirectives=true"
        "&_edgForm.showIfEmpty=false&_edgForm.darkMode=false"
    )
    await page.goto(target_url)
    await page.wait_for_load_state("networkidle")

    counters = await page.evaluate('''() => {
        const elements = Array.from(document.querySelectorAll('li.counter[data-behaviour="counter"]'));
        return elements.map(el => {
            const name = el.querySelector('span.name')?.innerText.trim() || "";
            const dataReceived = el.querySelector('span.count')?.getAttribute("data-received") || "0";
            return { name, dataReceived };
        });
    }''')

    # Process the extracted data:
    # If an element has data-received == 0 and the next element's name is "Navnet",
    # remove the next element and add its data-received value to the current element.
    merged_counters = []
    i = 0
    while i < len(counters):
        current = counters[i]
        current_val = int(current['dataReceived'])
        if current_val == 0 and i + 1 < len(counters) and counters[i+1]['name'] == "Navnet":
            next_val = int(counters[i+1]['dataReceived'])
            current['dataReceived'] = str(current_val + next_val)
            merged_counters.append(current)
            i += 2
        else:
            merged_counters.append(current)
            i += 1

    return merged_counters

async def scrape_week():
    async with async_playwright() as p:
        browser = await p.chromium.launch(
            headless=True,
            args=["--no-sandbox", "--disable-dev-shm-usage"]
        )
        page = await browser.new_page()

        # Log in
        login_url = "https://app.di.no/app/api/login"
        await page.goto(login_url)
        await page.wait_for_selector('input[name="username"]')
        await page.wait_for_selector('input[name="password"]')
        await page.fill('input[name="username"]', "lucas")
        await page.fill('input[name="password"]', "lucas")
        await page.click('input[name="login"]')
        await page.wait_for_load_state("networkidle")

        # Loop over a week ahead.
        # Here, we start from "15.02.2025" (as in your original example) and scrape 7 consecutive days.
        start_date = datetime.strptime("15.02.2025", "%d.%m.%Y")
        week_data = {}
        for day_offset in range(7):
            current_date = start_date + timedelta(days=day_offset)
            date_str = current_date.strftime("%d.%m.%Y")
            day_data = await scrape_counters_for_date(page, date_str)
            week_data[date_str] = day_data

        try:
            await browser.close()
        except TargetClosedError:
            pass

        return week_data

results = await scrape_week()

# Print the results with each day's data on new lines in one print call.
output_lines = []
for date, data in results.items():
    output_lines.append(f"Date: {date}")
    for item in data:
        output_lines.append(str(item))
    output_lines.append("")  # blank line between dates

print("\n".join(output_lines))


Date: 15.02.2025
{'name': 'ADR', 'dataReceived': '85'}
{'name': 'AFM', 'dataReceived': '1'}
{'name': 'DAGN', 'dataReceived': '1'}
{'name': 'DB', 'dataReceived': '2'}
{'name': 'DN', 'dataReceived': '2'}
{'name': 'FIA', 'dataReceived': '2'}
{'name': 'KLK', 'dataReceived': '12'}
{'name': 'LIMT', 'dataReceived': '12'}
{'name': 'PAKKE', 'dataReceived': '15'}
{'name': 'VL', 'dataReceived': '3'}

Date: 16.02.2025

Date: 17.02.2025
{'name': 'ADR', 'dataReceived': '65'}
{'name': 'AFM', 'dataReceived': '1'}
{'name': 'FIA', 'dataReceived': '1'}
{'name': 'KLK', 'dataReceived': '6'}
{'name': 'PAKKE', 'dataReceived': '10'}
{'name': 'VL', 'dataReceived': '2'}

Date: 18.02.2025
{'name': 'ADR', 'dataReceived': '65'}
{'name': 'AFM', 'dataReceived': '1'}
{'name': 'BOMAG', 'dataReceived': '2'}
{'name': 'BYGFA', 'dataReceived': '1'}
{'name': 'DAGN', 'dataReceived': '1'}
{'name': 'FIA', 'dataReceived': '1'}
{'name': 'FOS', 'dataReceived': '1'}
{'name': 'FUGLH', 'dataReceived': '2'}
{'name': 'HELSE', 'dataRe

In [ ]:
# Print the results in a more readable format
output_lines = []
for date, data in results.items():
    output_lines.append(f"Date: {date}")
    if data:
        for item in data:
            output_lines.append(f"  {item['name']}: {item['dataReceived']}")
    else:
        output_lines.append("  No data")
    output_lines.append("")  # Blank line between dates

print("\n".join(output_lines))


Date: 15.02.2025
  ADR: 85
  AFM: 1
  DAGN: 1
  DB: 2
  DN: 2
  FIA: 2
  KLK: 12
  LIMT: 12
  PAKKE: 15
  VL: 3

Date: 16.02.2025
  No data

Date: 17.02.2025
  ADR: 65
  AFM: 1
  FIA: 1
  KLK: 6
  PAKKE: 10
  VL: 2

Date: 18.02.2025
  ADR: 65
  AFM: 1
  BOMAG: 2
  BYGFA: 1
  DAGN: 1
  FIA: 1
  FOS: 1
  FUGLH: 2
  HELSE: 5
  HIT: 1
  HUFLI: 5
  KLK: 6
  NAM: 1
  SEH: 2
  TRAV: 1
  VL: 2

Date: 19.02.2025
  ADR: 65
  AFJ: 3
  AFM: 1
  FIA: 1
  KLK: 6
  NHAG: 4
  VL: 2

Date: 20.02.2025
  ADR: 65
  AFM: 1
  ALRS: 2
  DAGN: 1
  FIA: 1
  GP: 1
  HIST: 1
  KK: 2
  KLK: 6
  MFØRE: 1
  NAM: 1
  TRAV: 1
  VI60: 11
  VL: 2

Date: 21.02.2025
  ADR: 85
  AFM: 1
  BNDB1: 2
  DN: 2
  DOT: 4
  FIA: 2
  FOS: 1
  HIT: 1
  KLK: 8
  MB: 2
  VL: 3



In [ ]:
import json
from google.colab import auth
auth.authenticate_user()
import gspread
import google.auth
from datetime import datetime, timedelta

# Get credentials and authorize gspread.
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

# Open (or create) the Google Sheet.
spreadsheet_name = "KWGT_Data"
try:
    sh = gc.open(spreadsheet_name)
except Exception:
    sh = gc.create(spreadsheet_name)

# Delete worksheets whose titles are dates more than 30 days old.
today = datetime.today()
one_month_ago = today - timedelta(days=60)
for ws in sh.worksheets():
    try:
        ws_date = datetime.strptime(ws.title, "%d.%m.%Y")
        if ws_date < one_month_ago:
            sh.del_worksheet(ws)
            print(f"Deleted worksheet: {ws.title}")
    except ValueError:
        # Skip worksheets whose titles are not in date format.
        continue

# For each date in results, update a separate worksheet.
for date, data in results.items():
    try:
        ws = sh.worksheet(date)
    except gspread.exceptions.WorksheetNotFound:
        ws = sh.add_worksheet(title=date, rows=100, cols=10)
    ws.clear()
    rows = [["Name", "DataReceived"]]
    if data:
        for item in data:
            rows.append([item["name"], item["dataReceived"]])
    else:
        rows.append(["No data", ""])
    ws.update("A1", rows)

print("Google Sheet updated. Each date's data is in its own worksheet. Publish the sheet (File > Publish to the web) to use the URL as your JSON API endpoint in KWGT.")


Deleted worksheet: 15.01.2024


<ipython-input-36-391cea014edb>:45: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  ws.update("A1", rows)


Google Sheet updated. Each date's data is in its own worksheet. Publish the sheet (File > Publish to the web) to use the URL as your JSON API endpoint in KWGT.


In [ ]:
import json
import requests
import gspread
import google.auth

# Authenticate & connect to Google Sheets
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

# Open Google Sheet
SPREADSHEET_NAME = "KWGT_Data"
sh = gc.open(SPREADSHEET_NAME)

# Dictionary to store data formatted for KWGT
data_to_send = {}

# Process each worksheet (each date has its own worksheet)
for ws in sh.worksheets():
    records = ws.get_all_records()

    # If no data, store an empty message
    if not records:
        data_to_send[f"day_names_{ws.title.replace('.', '.')}"] = "No data available"
        data_to_send[f"day_counts_{ws.title.replace('.', '.')}"] = "No data available"
        continue

    # Extract names and counts separately
    product_names = "\n".join(item["Name"] for item in records)
    product_counts = "\n".join(str(item["DataReceived"]) for item in records)

    # Use the date as the key (formatted to remove dots for KWGT compatibility)
    kwgt_key_names = f"day_names_{ws.title.replace('.', '.')}"
    kwgt_key_counts = f"day_counts_{ws.title.replace('.', '.')}"

    # Store names and counts separately
    data_to_send[kwgt_key_names] = product_names
    data_to_send[kwgt_key_counts] = product_counts

# Replace with your actual KWGT App Token
KWGT_APP_TOKEN = "GfJ4UKAA5Qu6QZiBniXG7x0:APA91bGMzDGhWaupxkSV2aPhEyC_wifunto8IToVDA4eycBZfiiWa8TOpxfl7CY8fsgzghQ6i8tYIdp3yOCFQ7klX0Qan3Wkj0rNVVNUDR_5lnpNKWQj8-8"

# Format the payload for KWGT
kwgt_payload = {
    "tokens": [KWGT_APP_TOKEN],
    "data": data_to_send
}

# Send the data to KWGT via HTTP POST
KWGT_URL = "https://api.kustom.rocks/msg"
response = requests.post(
    KWGT_URL,
    headers={"Content-Type": "application/json"},
    data=json.dumps(kwgt_payload)
)

# Debugging Output in Colab
print("\nKWGT Response Code:", response.status_code)
print("KWGT Response Text:", response.text)
print("\nData Sent to KWGT:")
for key, value in data_to_send.items():
    print(f"{key}:")
    print(value)
    print("-" * 30)



KWGT Response Code: 200
KWGT Response Text: {}

Data Sent to KWGT:
day_names_15.02.2025:
ADR
AFM
DAGN
DB
DN
FIA
KLK
LIMT
PAKKE
VL
------------------------------
day_counts_15.02.2025:
85
1
1
2
2
2
12
12
15
3
------------------------------
day_names_16.02.2025:
No data
------------------------------
day_counts_16.02.2025:

------------------------------
day_names_17.02.2025:
ADR
AFM
FIA
KLK
PAKKE
VL
------------------------------
day_counts_17.02.2025:
65
1
1
6
10
2
------------------------------
day_names_18.02.2025:
ADR
AFM
BOMAG
BYGFA
DAGN
FIA
FOS
FUGLH
HELSE
HIT
HUFLI
KLK
NAM
SEH
TRAV
VL
------------------------------
day_counts_18.02.2025:
65
1
2
1
1
1
1
2
5
1
5
6
1
2
1
2
------------------------------
day_names_19.02.2025:
ADR
AFJ
AFM
FIA
KLK
NHAG
VL
------------------------------
day_counts_19.02.2025:
65
3
1
1
6
4
2
------------------------------
day_names_20.02.2025:
ADR
AFM
ALRS
DAGN
FIA
GP
HIST
KK
KLK
MFØRE
NAM
TRAV
VI60
VL
------------------------------
day_counts_20.02.202

In [ ]:
import gspread
import google.auth
from datetime import datetime

# Authenticate & connect to Google Sheets
creds, _ = google.auth.default()
gc = gspread.authorize(creds)

# Open (or create) the log sheet
SPREADSHEET_NAME = "Colab_Execution_Log"
try:
    sh = gc.open(SPREADSHEET_NAME)
except gspread.exceptions.SpreadsheetNotFound:
    sh = gc.create(SPREADSHEET_NAME)

worksheet = sh.sheet1  # Use the first worksheet

# Log current execution time
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
worksheet.append_row([now, "Colab executed"])

print("Execution logged successfully!")

Execution logged successfully!
